# 💧 VQE-Based Bond Angle Optimization of Water (H₂O)

This notebook implements the **Variational Quantum Eigensolver (VQE)** to determine the **optimal bond angle of the water molecule (H₂O)** by minimizing its ground state energy across a range of geometries. The simulation is executed on a **noiseless quantum device** using PennyLane’s quantum chemistry module.

### Key Features:
- **Geometry Generator**: Constructs H₂O molecular geometries parametrized by bond angle (in degrees) with fixed bond length.
- **VQE Energy Evaluation**:
  - Applies a **UCCSD-style ansatz** with both single and double excitations.
  - Optimizes the ansatz parameters for each bond angle using the **Gradient Descent optimizer**.
- **Angle Sweep**: Evaluates the ground state energy at discrete H–O–H angles (from 100° to 109°).
- **Result Visualization**:
  - Plots energy vs. bond angle to identify the minimum.
  - Outputs the **optimal angle** and its corresponding **minimum energy**.

### Purpose:
To estimate the **equilibrium bond angle** of water using a quantum variational approach, enabling quantum simulation-driven geometry optimization — a key step in **ab initio quantum chemistry** using near-term quantum devices.

In [ ]:
import pennylane as qml                  # Quantum circuit builder and device management
from pennylane import numpy as np        # Not regular NumPy, but similar and supports automatic differentiation
from pennylane import qchem              # Quantum chemistry module used to define molecule Hamiltonians
from pennylane.qchem import excitations  # Single and double excitations used in the UCCSD (Unitary Coupled Cluster Singles and Doubles) ansatz
import matplotlib.pyplot as plt
import sys, os, json
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))
from src.vqe_utils import excitation_ansatz, get_optimizer, set_seed, make_run_config_dict, run_signature, find_existing_run, save_run_record, ensure_dirs, build_run_filename
from src.vqe_utils import RESULTS_DIR, IMG_DIR

ensure_dirs()   # Creates results/ and images/, if missing

# Build config/signature for exact-match reuse
ansatz_desc = "UCCSD"
optimizer_name = "Adam"

symbols = ["O", "H", "H"]
basis = "STO-3G"
electrons = 10


In [ ]:
def water_geometry(angle_deg, bond_length=0.9584):
    # Generate coordinates for water with given angle in degrees (bond length in Å)
    angle_rad = np.deg2rad(angle_deg)  # Convert to radians
    x = bond_length * np.sin(angle_rad / 2)
    z = bond_length * np.cos(angle_rad / 2)
    coordinates = np.array([
        [0.0, 0.0, 0.0],  # Oxygen
        [x, 0.0, z],      # Hydrogen 1
        [-x, 0.0, z]      # Hydrogen 2
    ])
    return coordinates


def run_vqe_with_cache(angle_deg, seed, stepsize=0.2, max_iterations=50):
    set_seed(seed)

    # Geometry & Hamiltonian for this angle
    coordinates = water_geometry(angle_deg)
    hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge=0, basis=basis)
    hf = qchem.hf_state(electrons, qubits)
    singles, doubles = excitations(electrons, qubits)

    # Cost function (UCCSD: singles + doubles)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def cost_fn(params):
        excitation_ansatz(
            params, wires=range(qubits), hf_state=hf,
            excitations=(singles, doubles), excitation_type="both"
        )
        return qml.expval(hamiltonian)

    cfg = make_run_config_dict(
        symbols=symbols,
        coordinates=coordinates,
        basis=basis,
        ansatz_desc=ansatz_desc,
        optimizer_name=optimizer_name,
        stepsize=stepsize,
        max_iterations=max_iterations,
        seed=seed,
    )

    # Human-readable file includes angle
    sig = run_signature(cfg)
    fname = build_run_filename(f"H2O_BondAngle_{angle_deg:.1f}deg", optimizer_name, seed, sig)
    existing = find_existing_run(sig)

    if existing:
        with open(existing) as f:
            rec = json.load(f)
        print(f"[reuse] angle {angle_deg:.1f}°: loaded {existing}")
        energy = rec["energies"]
        # Params for downstream use (handle old/new schema)
        if "final_params" in rec:
            params = np.array(rec["final_params"], requires_grad=True)
        elif "params_by_step" in rec and rec["params_by_step"]:
            params = np.array(rec["params_by_step"][-1], requires_grad=True)
        else:
            print("[reuse] no params stored; recomputing.")
            existing = None

    if not existing:
        # Compute results if this configuration has NOT been ran before
        params = np.zeros(len(singles) + len(doubles), requires_grad=True)
        opt = get_optimizer(optimizer_name, stepsize=stepsize)
        energy = [cost_fn(params)]
        for _ in range(max_iterations):
            params, e_prev = opt.step_and_cost(cost_fn, params)
            energy.append(e_prev)
        energy[-1] = float(cost_fn(params))

        run_record = {
            "molecule": "H2O",
            "symbols": symbols,
            "geometry": np.array(coordinates).astype(float).tolist(),
            "basis": basis,
            "electrons": electrons,
            "num_wires": int(qubits),
            "ansatz": ansatz_desc,
            "optimizer": {
                "name": optimizer_name,
                "stepsize": float(stepsize),
                "iterations_planned": int(max_iterations),
                "iterations_ran": int(len(energy) - 1),
            },
            "seed": int(seed),
            "energies": [float(e) for e in np.array(energy).flatten()],
            "final_params": [float(x) for x in np.array(params).flatten()],
            "config_hash": str(sig),
            "metadata": {"angle_deg": float(angle_deg)},
        }

        persisted = save_run_record(fname, run_record)
        print(f"\n[saved] {optimizer_name}: {fname}")
        print(f"[mirrored] {persisted}")

    return float(energy[-1]), energy


In [ ]:
# Mutliple seeds for averaging
seeds = np.arange(0, 3)

# Define the angle range and initialize the energy
angles = np.linspace(100, 109, 5)  # Bond angles (in degrees) to test

energy_means, energy_stds = [], []

for angle in angles:
    print(f"Running VQE for angle {angle:.1f}° across {len(seeds)} seeds")

    results = []
    for seed in seeds:
        final_E, _curve = run_vqe_with_cache(angle, seed=seed, stepsize=0.2, max_iterations=20)
        results.append(final_E)

    energy_means.append(np.mean(results))
    energy_stds.append(np.std(results))

# Convert to arrays
energy_means = np.array(energy_means)
energy_stds = np.array(energy_stds)

# Plot the energy vs. angle
plt.errorbar(angles, energy_means, yerr=energy_stds, fmt="o-", capsize=4)
plt.xlabel("H–O–H Angle (°)")
plt.ylabel("Ground State Energy (Ha)")
plt.title("H₂O VQE: Energy vs. Bond Angle (Noiseless, UCCSD)")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{IMG_DIR}/H2O_Bond_Angle_Scan.png", dpi=300)
plt.show()


In [ ]:
# Print the optimum bond-angle and the corresponding energy
opt_angle = angles[np.argmin(energy_means)]
print(f"Minimum mean energy: {energy_means.min():.6f} ± {energy_stds[np.argmin(energy_means)]:.6f} Ha")
print(f"Optimal angle: {opt_angle:.2f}°")
